In [ ]:
from sklearn.datasets import fetch_20newsgroups
import cudf
import cupy as cp
import pandas as pd
from cuBERTopic import gpu_BERTopic
from embedding_extraction import create_embeddings
import rmm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
rmm.reinitialize(pool_allocator=True,initial_pool_size=5e+9)
data = fetch_20newsgroups(subset='all')['data']
# test_dataset = cudf.read_csv('/raid/mayanka/abcnews-date-text.csv')
# data = test_dataset['headline_text']

In [ ]:
len(data)

In [ ]:
gpu_topic = gpu_BERTopic()
documents = cudf.DataFrame(
    {"Document": data, "ID": cp.arange(len(data)), "Topic": None}
)

In [ ]:
%%time
# Extract embeddings
embeddings = create_embeddings(
    documents.Document
)

In [ ]:
%%time
# Dimensionality Reduction
umap_embeddings = gpu_topic.reduce_dimensionality(embeddings)
del embeddings

In [ ]:
%%time
# Cluster UMAP embeddings with HDBSCAN
documents, probabilities = gpu_topic.clustering_hdbscan(
    umap_embeddings,
    documents
)
del umap_embeddings

In [ ]:
%%time
# Topic representation
tf_idf, count, docs_per_topics_topics = gpu_topic.create_topics(
    documents
)
top_n_words, name_repr = gpu_topic.extract_top_n_words_per_topic(
    tf_idf, count, docs_per_topics_topics, n=30
)

gpu_topic.topic_sizes_df["Name"] = gpu_topic.topic_sizes_df["Topic"].map(
    name_repr
)

In [ ]:
## Running the end-to-end pipeline to extract topics

In [ ]:
%%time
gpu_topic_pipeline = gpu_BERTopic()
topics_gpu, probs_gpu = gpu_topic_pipeline.fit_transform(data)

In [ ]:
%%time
gpu_topic_pipeline.get_topic_info()

In [ ]:
%%time
gpu_topic_pipeline.get_topic(0)